# Graph Neural Network for Knowledge Graph Embedding

## Install dependencies

In [62]:
%%bash
pip3 install torch torchvision

  Using cached torch-2.0.1-cp310-none-macosx_11_0_arm64.whl (55.8 MB)
  Using cached torchvision-0.15.2-cp310-cp310-macosx_11_0_arm64.whl (1.4 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 22.3 MB/s eta 0:00:00
  Using cached networkx-3.1-py3-none-any.whl (2.1 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 25.6 MB/s eta 0:00:00
  Using cached typing_extensions-4.7.1-py3-none-any.whl (33 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-ai 2.0.0 requires typing-extensions==4.5.0, but you have typing-extensions 4.7.1 which is incompatible.
jupyter-ai-magics 2.0.0 requires typing-extensions==4.5.0, but you have typing-extensions 4.7.1 which is incompatible.


In [73]:
%%bash
pip install git+https://github.com/awslabs/dgl-ke.git#subdirectory=python
pip install ogb dgl==0.9

  Cloning https://github.com/awslabs/dgl-ke.git to /private/var/folders/4s/d4nr1sg91ps1k3qz00h28w_r0000gp/T/pip-req-build-8oz4vr6h


  Running command git clone --filter=blob:none --quiet https://github.com/awslabs/dgl-ke.git /private/var/folders/4s/d4nr1sg91ps1k3qz00h28w_r0000gp/T/pip-req-build-8oz4vr6h


  Resolved https://github.com/awslabs/dgl-ke.git to commit 43875a57a721d62396d05235226b13e6c2db1a2a
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 4.8 MB/s eta 0:00:001
  Attempting uninstall: dgl
    Found existing installation: dgl 1.1.2.post1
    Uninstalling dgl-1.1.2.post1:
      Successfully uninstalled dgl-1.1.2.post1


## Prepare Data

In [66]:
%%bash
wget https://s3.us-west-2.amazonaws.com/dgl-data/dataset/DRKG/drkg.tar.gz -O test/drkg.tar.gz
tar -xvzf test/drkg.tar.gz -C test/

--2023-09-04 17:30:02--  https://s3.us-west-2.amazonaws.com/dgl-data/dataset/DRKG/drkg.tar.gz
Resolving s3.us-west-2.amazonaws.com (s3.us-west-2.amazonaws.com)... 52.92.210.168, 52.92.139.136, 52.92.179.24, ...
Connecting to s3.us-west-2.amazonaws.com (s3.us-west-2.amazonaws.com)|52.92.210.168|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 216650245 (207M) [application/x-tar]
Saving to: ‘test/drkg.tar.gz’

     0K .......... .......... .......... .......... ..........  0%  618K 5m42s
    50K .......... .......... .......... .......... ..........  0%  684K 5m26s
   100K .......... .......... .......... .......... ..........  0% 14.9M 3m42s
   150K .......... .......... .......... .......... ..........  0%  704K 4m1s
   200K .......... .......... .......... .......... ..........  0% 21.1M 3m15s
   250K .......... .......... .......... .......... ..........  0% 22.4M 2m44s
   300K .......... .......... .......... .......... ..........  0% 2.00M 2m35s
   350K ...

In [77]:
import pandas as pd
import numpy as np

drkg_file = "test/drkg.tsv"
df = pd.read_csv(drkg_file, sep="\t", header=None)
triples = df.values.tolist()

In [78]:
num_triples = len(triples)
num_triples

5874261

In [79]:
import os 
# Please make sure the output directory exist.
seed = np.arange(num_triples)
np.random.shuffle(seed)

train_cnt = int(num_triples * 0.9)
valid_cnt = int(num_triples * 0.05)
train_set = seed[:train_cnt]
train_set = train_set.tolist()
valid_set = seed[train_cnt:train_cnt+valid_cnt].tolist()
test_set = seed[train_cnt+valid_cnt:].tolist()

os.makedirs("test/train", exist_ok=True)

with open("test/train/drkg_train.tsv", 'w+') as f:
    for idx in train_set:
        f.writelines("{}\t{}\t{}\n".format(triples[idx][0], triples[idx][1], triples[idx][2]))
        
with open("test/train/drkg_valid.tsv", 'w+') as f:
    for idx in valid_set:
        f.writelines("{}\t{}\t{}\n".format(triples[idx][0], triples[idx][1], triples[idx][2]))

with open("test/train/drkg_test.tsv", 'w+') as f:
    for idx in test_set:
        f.writelines("{}\t{}\t{}\n".format(triples[idx][0], triples[idx][1], triples[idx][2]))

print(len(train_set), len(valid_set), len(test_set))

5286834 293713 293714


## Train models with different hyperparameters

In [93]:
%%bash
DGLBACKEND=pytorch dglke_train --dataset drkg --model_name TransE_l2 --batch_size 2048 --neg_sample_size 256 --hidden_dim 400 --gamma 12.0 --lr 0.1 --max_step 100000 --log_interval 1000 --batch_size_eval 16 --neg_sample_size_eval 10000 -adv --regularization_coef 1.00E-07 --data_path ./test/train --data_files drkg_train.tsv drkg_valid.tsv drkg_test.tsv --format raw_udd_hrt --save_path ./test/model --num_thread 20

/Users/jy006/miniconda3/envs/network-medicine/lib/python3.10/site-packages/dgl/_deprecate/graph.py:1023: DGLWarning: multigraph will be deprecated.DGL will treat all graphs as multigraph in the future.
  dgl_warning("multigraph will be deprecated." \


Reading train triples....
Finished. Read 5286834 train triples.
Reading valid triples....
Finished. Read 293713 valid triples.
Reading test triples....
Finished. Read 293714 test triples.
|Train|: 5286834
Total initialize time 7.581 seconds
[proc 0][Train](1000/100000) average pos_loss: 0.5619124862884847
[proc 0][Train](1000/100000) average neg_loss: 0.6707942681014538
[proc 0][Train](1000/100000) average loss: 0.616353376954794
[proc 0][Train](1000/100000) average regularization: 0.029927969855024
[proc 0][Train] 1000 steps take 24.919 seconds
[proc 0]sample: 1.547, forward: 9.443, backward: 9.998, update: 3.928
[proc 0][Train](2000/100000) average pos_loss: 0.4474902266860008
[proc 0][Train](2000/100000) average neg_loss: 0.5624074468612671
[proc 0][Train](2000/100000) average loss: 0.504948835581541
[proc 0][Train](2000/100000) average regularization: 0.033087124606594444
[proc 0][Train] 1000 steps take 25.786 seconds
[proc 0]sample: 1.485, forward: 9.749, backward: 10.452, update:

### Model 1: TransE

### Model 2: TransR